In [1]:
import random

locations = [(1, 5), (2, 3), (4, 8), (6, 1), (7, 4), (8, 2)]
cars = 2 #amount of required subsets
generations = 5
depot = (0, 0)
population_size = 10


In [8]:
# Problem: Traveling Salesman Problem
    

def create_individual(n_cars):
    """
    Creates a random individual for the Traveling Salesman Problem.
    
    Returns:
    list: A permutation of city indices representing a tour.
    """
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

    
    return list(split(random.sample(range(len(locations)), len(locations)), n_cars))

def fitness(individual):
    """
    Calculates the fitness of an individual.
    
    The fitness is the inverse of the total distance of the tour.
    Shorter tours have higher fitness values.
    
    Args:
    individual (list): A permutation of city indices representing a tour.
    
    Returns:
    float: The inverse of the total distance of the tour.
    """
    total_distance = sum(
        ((locations[individual[i]][0] - locations[individual[i-1]][0])**2 + 
         (locations[individual[i]][1] - locations[individual[i-1]][1])**2)**0.5
        for i in range(len(individual))
    )
    return 1 / total_distance

def crossover(parent1, parent2):
    """
    Performs Order Crossover (OX) between two parents.
    
    Args:
    parent1 (list): First parent's tour.
    parent2 (list): Second parent's tour.
    
    Returns:
    tuple: A tuple containing:
           - The child resulting from the crossover (list)
           - The start and end indices of the crossover section (tuple)
    """
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    pointer = end
    for location in parent2:
        if location not in child:
            child[pointer] = location
            pointer = (pointer + 1) % len(child)
    return child, (start, end)

def mutate(individual):
    """
    Mutates an individual by swapping two cities.
    
    Args:
    individual (list): The tour to mutate.
    
    Returns:
    list: The mutated tour.
    """
    if random.random() < 0.1:  # 10% mutation rate for demonstration
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

def visualize_individual(individual, fitness_value):
    """
    Creates a string representation of an individual.
    
    Args:
    individual (list): A tour represented as a list of city indices.
    fitness_value (float): The fitness value of the individual.
    
    Returns:
    str: A string representation of the tour and its fitness.
    """
    route = " -> ".join(map(str, individual))
    return f"{route} Fitness: {fitness_value:.6f}"

def genetic_algorithm():
    """
    Runs the genetic algorithm for the Traveling Salesman Problem.
    
    This function initializes a population, evolves it over several generations,
    and prints the state of the population at each step.
    """
    population = [create_individual(cars) for _ in range(population_size)]
    
    for generation in range(generations):
        print(f"\nGeneration {generation}:")
        for i, ind in enumerate(population):
            print(f"Individual {i}: {visualize_individual(ind, fitness(ind))}")
        
        new_population = []
        
        print("\nCrossover and Mutation:")
        for i in range(0, population_size, 2):
            parent1, parent2 = random.sample(population, 2)
            child1, (start, end) = crossover(parent1, parent2)
            child2, _ = crossover(parent2, parent1)
            
            print(f"\nParent 1: {visualize_individual(parent1, fitness(parent1))}")
            print(f"Parent 2: {visualize_individual(parent2, fitness(parent2))}")
            print(f"Crossover range: {start} to {end}")
            print(f"Child 1:  {visualize_individual(child1, fitness(child1))}")
            
            mutated_child1 = mutate(child1)
            print(f"Mutated: {visualize_individual(mutated_child1, fitness(mutated_child1))}")
            
            new_population.extend([mutated_child1, mutate(child2)])
        
        population = new_population
    
    print("\nFinal Population:")
    for i, ind in enumerate(population):
        print(f"Individual {i}: {visualize_individual(ind, fitness(ind))}")

if __name__ == "__main__":
    genetic_algorithm()


Generation 0:


TypeError: list indices must be integers or slices, not list

In [ ]:
create_